In [1]:
import os
from functions import extract_text_from_pdf , extract_multiline_field , clean_text
import pandas as pd
import joblib

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\guruk\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\guruk\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
pdf_path = input("Enter the resume PDF path: ").strip()
pdf_data = []

if pdf_path.endswith(".pdf"):
    try:
        text = extract_text_from_pdf(pdf_path)
        row = {
            "Education": extract_multiline_field("Education", text),
            "Skills": extract_multiline_field("Skills", text),
            "Projects": extract_multiline_field("Projects", text),
            "Certifications": extract_multiline_field("Certifications", text)
        }
        pdf_data.append(row)
        print("✅ Extraction DONE")
    except Exception as e:
        print("❌ Error reading the PDF:", e)
else:
    print("⚠️ You have entered an invalid file path or non-PDF.")


✅ Extraction DONE


In [3]:
columns = ["Education", "Skills", "Projects", "Certifications", "Experience", "Achievements"]

# Grab the first row (dictionary) from the list
raw_row = pdf_data[0]  # If it's a list with one dictionary

# Make sure all 6 columns exist
pdf_row = {col: raw_row.get(col, '') for col in columns}

# Create DataFrame and append the row
df = pd.DataFrame(columns=columns)
df = pd.concat([df, pd.DataFrame([pdf_row])], ignore_index=True)

# Clean and combine text
for col in df.columns:
    df[col] = df[col].fillna('').apply(clean_text)

df["combined_text"] = (
    df["Education"] + " " +
    df["Skills"] + " " +
    df["Projects"] + " " +
    df["Certifications"] + " " +
    df["Experience"] + " " +
    df["Achievements"]
)


In [4]:
# Replace with your actual paths if they are in a different folder
model = joblib.load("resume_model.pkl")
vectorizer = joblib.load("vectorizer.pkl")


In [5]:
X = vectorizer.transform(df['combined_text'])

# Predict
predicted_score = model.predict(X)[0]

print(f"🎯 Predicted Resume Score: {predicted_score:.2f} / 100")

🎯 Predicted Resume Score: 8.08 / 100
